In [1]:
# pip install mysql


In [2]:
import pandas as pd
import pymysql

tweetSchema= {'fechaCreacion': {'format': '%Y-%m-%d %H:%M:%S'}}

myconnection = pymysql.connect(host = 'localhost', user = 'root',password = 'root',db = 'dbtwitter', charset='utf8')

data_db = None
try:
    query = """SELECT * FROM tweet"""
    data_db = pd.read_sql(query, con = myconnection, parse_dates = tweetSchema)
finally:
    myconnection.close()

data_db.head()

,id,idUsuario,fechaCreacion,texto
0,1062491062751686656,226802593,2018-11-13 23:43:04,Como cuando ahora también tiene que prepararte...
1,1062491070980935682,1020688128250646528,2018-11-13 23:43:06,Sres. @AsambleaEcuador no arriesguen más la sa...
2,1062491474594476039,353815647,2018-11-13 23:44:42,Comencemos esta experiencia!! #psicologosenelm...
3,1062491477002084352,561395391,2018-11-13 23:44:42,@galarzaramiro1 @AdrianPalacioJ @VillaFernando...


In [3]:
df_tweet_texto = data_db["texto"]
# df_tweet_texto

In [4]:
import string
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [5]:
stop_words = stopwords.words('spanish')
stop_words.extend(["rt", "aun", "oe"])
exclude = string.punctuation
exclude = exclude + "¿"

lemma = SnowballStemmer('spanish')

print(stop_words)
print(exclude)

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosostros', 'vosostras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 

In [6]:
import re

##def remove_number(list_text):
##    return ''.join([i for i in text if not i.isdigit()])

def remove_three_dots(list_text):
    return [re.sub(r"[a-zA-Z]+(\……|\…)$", " ", texto) for texto in list_text]

def remove_url(list_text):
    return [re.sub(r"http\S+", "", texto).strip() for texto in list_text]

def remove_breakline(list_text):
    return [re.sub('\s+', ' ', texto) for texto in list_text]

def remove_single_quotes(list_text):
    return [re.sub("\'", "", texto) for texto in list_text]

In [12]:
data = data_db.texto.values.tolist()
print(data)

['Como cuando ahora también tiene que prepararte psicológicamente para el torneo #marcianogaming #PUBGMOBILE… https://t.co/buM1Bn6dqE', 'Sres. @AsambleaEcuador no arriesguen más la salud de los jóvenes mediante el uso “medicinal” de marihuana. Sin estu… https://t.co/u3DKINYIcf', 'Comencemos esta experiencia!! #psicologosenelmundo #psicologovenezolano #reclutamientoyseleccion… https://t.co/EGMyHDcGij', '@galarzaramiro1 @AdrianPalacioJ @VillaFernando_ Debería pedirle el  crédito a AP; tienen para darnos fácil unos 20B… https://t.co/trwLssbhfC']


In [13]:
import gensim

txt_free_url = remove_url(data)
txt_free_threedot = remove_three_dots(txt_free_url)
txt_free_breakline = remove_breakline(txt_free_threedot)
txt_free_singlequotes = remove_single_quotes(txt_free_breakline)

def texto_a_palabras(texto:str):
    for sentencia in texto:
        yield(gensim.utils.simple_preprocess(str(sentencia), deacc=True))
        
data_palabras = list(texto_a_palabras(txt_free_singlequotes))

print(data_palabras)

[['como', 'cuando', 'ahora', 'tambien', 'tiene', 'que', 'prepararte', 'para', 'el', 'torneo', 'marcianogaming'], ['sres', 'asambleaecuador', 'no', 'arriesguen', 'mas', 'la', 'salud', 'de', 'los', 'jovenes', 'mediante', 'el', 'uso', 'medicinal', 'de', 'marihuana', 'sin'], ['comencemos', 'esta', 'experiencia'], ['galarzaramiro', 'adrianpalacioj', 'villafernando_', 'deberia', 'pedirle', 'el', 'credito', 'ap', 'tienen', 'para', 'darnos', 'facil', 'unos']]


In [16]:
bigram = gensim.models.Phrases(data_palabras, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_palabras], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/home/jkn/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [29]:
print(bigram_mod[data_palabras[2]])

['comencemos', 'esta', 'experiencia']


In [34]:
import spacy
# python -m spacy download es

def remove_stopwords(texts):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

data_words_nostops = remove_stopwords(data_palabras)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('es', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized)

[['ahora', 'tambien', 'prepararte', 'tornear', 'marcianogaming'], ['sres', 'asambleaecuador', 'arriesgar', 'mas', 'salud', 'jovenes', 'usar', 'medicinal', 'marihuana'], ['comenzar', 'experiencia'], ['galarzaramiro', 'adrianpalacioj', 'villafernando', '_', 'deberia', 'pedirle', 'credito', 'ap', 'darnos', 'facil']]


In [35]:
from gensim import corpora
 
diccionario = corpora.Dictionary(data_lemmatized)
#Corpus
texto = data_lemmatized

In [38]:
corpus = [diccionario.doc2bow(doc) for doc in texto]

print(data_words_bigrams[2])
print(corpus[2])

['comencemos', 'experiencia']
[(14, 1), (15, 1)]


In [39]:
print (data_words_bigrams[1])
print (corpus[1])

['sres', 'asambleaecuador', 'arriesguen', 'mas', 'salud', 'jovenes', 'mediante', 'uso', 'medicinal', 'marihuana']
[(5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]


In [42]:
[[(diccionario[id], freq) for id, freq in cp] for cp in corpus[0:3]]

[[('ahora', 1),
  ('marcianogaming', 1),
  ('prepararte', 1),
  ('tambien', 1),
  ('tornear', 1)],
 [('arriesgar', 1),
  ('asambleaecuador', 1),
  ('jovenes', 1),
  ('marihuana', 1),
  ('mas', 1),
  ('medicinal', 1),
  ('salud', 1),
  ('sres', 1),
  ('usar', 1)],
 [('comenzar', 1), ('experiencia', 1)]]

In [43]:
print(len(list(diccionario.values())))

26


In [44]:
#Correr y entrenar el modelo LDA sobre la matriz de términos.
lda_model = gensim.models.LdaModel(corpus,
                                   num_topics=20,
                                   id2word = diccionario,
                                   random_state=100,
                                   passes=10,
                                   update_every=1,
                                   chunksize=100,
                                   alpha='auto',
                                   per_word_topics=True)

In [45]:
for idx, topic in lda_model.print_topics():
    print('Topic: {} Word: {}\n'.format(idx, topic))

Topic: 0 Word: 0.038*"credito" + 0.038*"deberia" + 0.038*"_" + 0.038*"adrianpalacioj" + 0.038*"ap" + 0.038*"comenzar" + 0.038*"darnos" + 0.038*"pedirle" + 0.038*"galarzaramiro" + 0.038*"sres"

Topic: 1 Word: 0.038*"credito" + 0.038*"deberia" + 0.038*"_" + 0.038*"adrianpalacioj" + 0.038*"ap" + 0.038*"comenzar" + 0.038*"darnos" + 0.038*"pedirle" + 0.038*"galarzaramiro" + 0.038*"sres"

Topic: 2 Word: 0.038*"credito" + 0.038*"deberia" + 0.038*"_" + 0.038*"adrianpalacioj" + 0.038*"ap" + 0.038*"comenzar" + 0.038*"darnos" + 0.038*"pedirle" + 0.038*"galarzaramiro" + 0.038*"sres"

Topic: 3 Word: 0.038*"credito" + 0.038*"deberia" + 0.038*"_" + 0.038*"adrianpalacioj" + 0.038*"ap" + 0.038*"comenzar" + 0.038*"darnos" + 0.038*"pedirle" + 0.038*"galarzaramiro" + 0.038*"sres"

Topic: 4 Word: 0.038*"credito" + 0.038*"deberia" + 0.038*"_" + 0.038*"adrianpalacioj" + 0.038*"ap" + 0.038*"comenzar" + 0.038*"darnos" + 0.038*"pedirle" + 0.038*"galarzaramiro" + 0.038*"sres"

Topic: 5 Word: 0.038*"credito" + 0.

In [61]:
# pip install pyLDAvis

import warnings
import pyLDAvis.gensim

warnings.filterwarnings('ignore')

pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(lda_model,corpus,diccionario)
vis

print(diccionario)

Dictionary(26 unique tokens: ['ahora', 'marcianogaming', 'prepararte', 'tambien', 'tornear']...)
